<a href="https://colab.research.google.com/github/yashica12/Neural-Networks/blob/master/Recommend_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install torch torchvision

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive/')


In [0]:
#importing data set 
path = '/content/drive/My Drive/Colab Notebooks/cnn/bm/bm/bm/'
movies = pd.read_csv(path + 'ml-1m/movies.dat',sep = '::' , header = None,engine='python' , encoding='latin-1')

In [0]:
users = pd.read_csv(path + 'ml-1m/users.dat',sep = '::' , header = None,engine='python' , encoding='latin-1')

In [0]:
ratings = pd.read_csv(path + 'ml-1m/ratings.dat',sep = '::' , header = None,engine='python' , encoding='latin-1')

In [0]:
#importing test and training set
training_set = pd.read_csv(path + 'ml-100k/u1.base' , delimiter='\t')

In [0]:
training_set =np.array(training_set,dtype='int')

In [0]:
test_set = pd.read_csv(path + 'ml-100k/u1.test' , delimiter='\t')
test_set = np.array(test_set,dtype='int')

In [0]:
# total users and movies in the data set 
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))



In [0]:
#converting data into an array 
def convert(data) : 
  new_data = []
  for i in range(1,nb_users + 1 ):
    id_movies = data[:,1][data[:,0] == i]
    print("MOVIES" , id_movies)
    id_ratings = data[:,2][data[:,0] == i]
    print("RATINGS" , id_ratings) 
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1 ] = id_ratings
    new_data.append(list(ratings))
  return new_data

  

In [36]:
training_set = convert(training_set)


MOVIES [  2   3   4   5   7   8   9  11  13  15  16  18  19  21  22  25  26  28
  29  30  32  34  35  37  38  40  41  42  43  45  46  48  50  52  55  57
  58  59  63  66  68  71  75  77  79  83  87  88  89  93  94  95  99 101
 105 106 109 110 111 115 116 119 122 123 124 126 127 131 133 135 136 137
 138 139 141 142 144 146 147 149 152 153 156 158 162 165 166 167 168 169
 172 173 176 178 179 181 182 187 191 192 194 195 197 198 199 203 204 205
 207 211 216 217 220 223 231 234 237 238 239 240 244 245 246 247 249 251
 256 257 261 263 268 269 270 271]
RATINGS [3 4 3 3 4 1 5 2 5 5 5 4 5 1 4 4 3 4 1 3 5 2 1 2 3 3 2 5 4 5 4 5 5 4 5 5 4
 5 2 4 4 3 4 4 4 3 5 4 5 5 2 4 3 2 2 4 5 1 5 5 3 5 3 4 5 2 5 1 4 4 3 5 1 3
 3 2 4 4 3 2 5 3 4 3 4 5 5 2 5 5 5 5 5 5 3 5 4 4 5 4 4 5 5 5 4 4 5 3 5 3 5
 3 3 5 1 4 2 4 4 3 2 2 5 1 4 4 4 4 1 1 5 5 5 2]
MOVIES [  1  10  14  25 100 111 127 237 242 255 258 269 272 273 274 275 276 277
 278 282 283 284 285 286 287 288 289 291 293 294 295 296 300 302 304 305
 306 309 310 3

In [37]:
test_set = convert(test_set)

MOVIES [ 10  12  14  17  20  23  24  27  31  33  36  39  44  47  49  51  53  54
  56  60  61  62  64  65  67  69  70  72  73  74  76  78  80  81  82  84
  85  86  90  91  92  96  97  98 100 102 103 104 107 108 112 113 114 117
 118 120 121 125 128 129 130 132 134 140 143 145 148 150 151 154 155 157
 159 160 161 163 164 170 171 174 175 177 180 183 184 185 186 188 189 190
 193 196 200 201 202 206 208 209 210 212 213 214 215 218 219 221 222 224
 225 226 227 228 229 230 232 233 235 236 241 242 243 248 250 252 253 254
 255 258 259 260 262 264 265 266 267 272]
RATINGS [3 5 5 3 4 4 3 2 3 4 2 4 5 4 3 4 3 3 4 5 4 3 5 4 3 3 3 4 3 1 4 1 4 5 5 4 3
 5 4 5 3 5 3 4 5 2 1 1 4 5 1 5 5 3 3 1 4 3 4 5 3 4 4 1 1 2 2 5 4 5 2 4 3 4
 4 4 3 5 5 5 5 5 3 5 4 4 4 3 3 5 4 5 3 3 5 4 5 4 4 4 2 4 3 3 1 5 4 5 2 3 4
 5 4 4 3 2 5 4 4 5 1 4 4 2 5 1 2 5 1 1 3 2 4 1 4 3]
MOVIES [ 13  19  50 251 257 279 280 281 290 292 297 298 299 301 303 307 308 312
 313 314 315 316]
RATINGS [4 3 5 5 4 4 3 3 3 4 4 3 4 4 4 3 3 3 5 1 1 5]
MOV

In [38]:
len(test_set)

943

In [0]:
training_set = torch.FloatTensor(training_set)

In [0]:
test_set = torch.FloatTensor(test_set)

In [0]:
#converting liked = 1 , not liked =0 , not rated = -1 
training_set[training_set == 0 ] = -1 
training_set[training_set == 1 ] = 0 
training_set[training_set == 2 ] = 0 
training_set[training_set >= 3 ] = 1

test_set[test_set == 0 ] = -1 
test_set[test_set == 1 ] = 0 
test_set[test_set == 2 ] = 0 
test_set[test_set >= 3 ] = 1

In [0]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
  
    
      
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))  
  